# SMB远程文件创建（SMB Create Remote File）

## 概述

| 编写作者 | 攻击矩阵 | 创建时间 | 修改时间 |
| ---- | ---- | ---- | ---- |
| G4rb3n | T1123 | 2021-4-8 | 2021-4-8 |

## 攻击场景

黑客通过访问注册表键值来查看麦克风信息。

## 攻击模拟

In [ ]:
msf5 exploit(multi/handler) > use post/multi/manage/record_mic
msf5 post(multi/manage/record_mic) > set SESSION 2
SESSION => 2
msf5 post(multi/manage/record_mic) > info

      Name: Multi Manage Record Microphone
    Module: post/multi/manage/record_mic
  Platform: Linux, OSX, Windows
      Arch: 
      Rank: Normal

Provided by:
  sinn3r <sinn3r@metasploit.com>

Compatible session types:
  Meterpreter

Basic options:
  Name      Current Setting  Required  Description
  ----      ---------------  --------  -----------
  DURATION  5                no        Number of seconds to record
  SESSION   2                yes       The session to run this module on.

Description:
  This module will enable and record your target's microphone. For 
  non-Windows targets, please use Java meterpreter to be able to use 
  this feature.

msf5 post(multi/manage/record_mic) > run

[*] 172.18.39.6 - 20%...
[*] 172.18.39.6 - 40%...
[*] 172.18.39.6 - 60%...
[*] 172.18.39.6 - 80%...
[*] 172.18.39.6 - 100%...
[*] 172.18.39.6 - Audio size: (55169 bytes)
[+] 172.18.39.6 - Audio recording saved: /home/msf/.msf4/loot/20200610025201_default_172.18.39.6_172.18.39.6.audi_358712.wav
[*] Post module execution completed
msf5 post(multi/manage/record_mic) >

## 威胁狩猎

### 狩猎原理

监控注册表项键值：HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\CapabilityAccessManager\ConsentStore\microphone\NonPackaged，来判断什么应用可能访问了麦克风。

### 初始化分析引擎

In [ ]:
from openhunt.mordorutils import *
spark = get_spark()

### 下载&加载Mordor日志集

In [ ]:
mordor_file = "https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/collection/host/msf_record_mic.zip"
registerMordorSQLTable(spark, mordor_file, "mordorTable")

[+] Processing a Spark DataFrame..
[+] DataFrame Returned !
[+] Temporary SparkSQL View: mordorTable 


### 日志集统计

In [ ]:
df = spark.sql(
'''
SELECT Hostname,Channel,EventID, Count(*) as count
FROM mordorTable
GROUP BY Hostname,Channel,EventID
ORDER BY count DESC
'''
)
df.show(truncate=False)

+-------------------------+------------------------------------+-------+-----+
|Hostname                 |Channel                             |EventID|count|
+-------------------------+------------------------------------+-------+-----+
|WORKSTATION6.mordor.local|Microsoft-Windows-Sysmon/Operational|12     |1635 |
|WORKSTATION6.mordor.local|Microsoft-Windows-Sysmon/Operational|10     |1464 |
|WORKSTATION6.mordor.local|Microsoft-Windows-Sysmon/Operational|13     |605  |
|WORKSTATION5.mordor.local|Microsoft-Windows-Sysmon/Operational|10     |404  |
|WORKSTATION6.mordor.local|Microsoft-Windows-Sysmon/Operational|7      |303  |
|MORDORDC.mordor.local    |Microsoft-Windows-Sysmon/Operational|7      |277  |
|MORDORDC.mordor.local    |Microsoft-Windows-Sysmon/Operational|12     |157  |
|WORKSTATION5.mordor.local|Microsoft-Windows-Sysmon/Operational|13     |145  |
|MORDORDC.mordor.local    |Security                            |4703   |119  |
|MORDORDC.mordor.local    |Security                 

### 使用规则匹配攻击事件

### 规则 I

```
—————————————————————————————————————————————————————————————————————————————————
|        日志类型         |    日志来源    |          行为场景         |   事件ID  |
---------------------------------------------------------------------------------
|        Registry         |    security   |   进程/用户访问了注册表项  |   4663   |
---------------------------------------------------------------------------------
|        Registry         |    security   |  进程/用户请求访问注册表项 |   4656   |
---------------------------------------------------------------------------------
|        Registry         |    security   |   进程/用户修改了注册表项  |   4657   |
—————————————————————————————————————————————————————————————————————————————————
```

筛选事件ID为5140（User accessed file share）的security事件。

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, EventID, ObjectName
FROM mordorTable
WHERE LOWER(Channel) = 'security'
  AND EventID IN (4656,4663,4657)
  AND LOWER(ObjectName) RLIKE '.*consentstore\\\\\\\microphone.*'
'''
)
df.show(10, False)

+------------------------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|@timestamp              |EventID|ObjectName                                                                                                                                                                       |
+------------------------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2020-06-10T02:51:56.437Z|4656   |\REGISTRY\MACHINE\SOFTWARE\Microsoft\Windows\CurrentVersion\CapabilityAccessManager\ConsentStore\microphone\NonPackaged\C:#Windows#System32#WindowsPowerShell#v1.0#powershell.exe|
|2020-06-10T02:51:56.437Z|4663   |\REGISTRY\MACHINE\SOFTWARE\Microsoft\Windows\CurrentVersion\CapabilityAccessManager\ConsentStore\microphone\NonPac

## 规则II

```
—————————————————————————————————————————————————————————————————————————————————
|        Registry         |     sysmon    |      进程创建注册表键值    |    12    |
---------------------------------------------------------------------------------
|        Registry         |     sysmon    |      进程创建注册表键值    |    13    |
---------------------------------------------------------------------------------
|        Registry         |     sysmon    |      进程创建注册表键值    |    14    |
—————————————————————————————————————————————————————————————————————————————————
```

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, EventID, TargetObject
FROM mordorTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
  AND EventID IN (12,13,14)
  AND LOWER(TargetObject) RLIKE '.*consentstore\\\\\\\microphone.*'
'''
)
df.show(10,False)

+------------------------+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|@timestamp              |EventID|TargetObject                                                                                                                                                                          |
+------------------------+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2020-06-10T02:51:55.390Z|12     |HKU\S-1-5-21-526538150-889687948-186688817-1106\Software\Microsoft\Windows\CurrentVersion\CapabilityAccessManager\ConsentStore\microphone\NonPackaged                                 |
|2020-06-10T02:51:55.390Z|13     |HKU\S-1-5-21-526538150-889687948-186688817-1106\Software\Microsoft\Windows\CurrentVersion\Capa